In [ ]:

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pyautogui as pg
import time
from tqdm import tqdm
import pandas as pd
from config import *
from functions import *

# Read in the list of names we will be adding to the CRM for each location
df =  pd.read_csv('email_list.csv')
actives = df['Name'].values.tolist()
actives = [h.title() for h in actives]

#getting connected. chrome_options used for a specific function in private version
response = requests.get(url)
chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

#locate login fields
username = driver.find_element( By.ID, 'user_email')
password = driver.find_element(By.ID, 'user_password')

# enter the login credentials
username.send_keys(u)
password.send_keys(pw)

# login
login_button = driver.find_element(By.NAME, 'commit')
login_button.click()

# begin navigation
dropdown = driver.find_element(By.ID, 'user-usurp')
dropdown.click()

# create a list of dropdown options and return the display text to enable simpler iteration
options = Select(driver.find_elements(By.ID, "user-userp")).text()

# cycle through each location navigating through menus
for i in options:
    location = driver.find_element(By.LINK_TEXT, i)
    location.click()
    sources = driver.find_element(By.LINK_TEXT, 'Sources')
    sources.click()
    time.sleep(1)
    # cycle through each potential individual in each clinic
    for g in tqdm(actives):
        search_referrer = driver.find_element(By.NAME, 'search_referrer')
        search_referrer.send_keys(g)
        pg.press('enter')
        time.sleep(1)
        # defined in function. uses pyautogui to locate interactions for elements stored in strange ways
        add_to_list()
        # clears the search bar for the next one
        search_referrer.clear()
    # preps for the next iteration
    dropdown = driver.find_element(By.CLASS_NAME, "dropdown-toggle")
    dropdown.click()  

